Задача: очистить и подготовить данные из файла Uber.csv

In [ ]:
import pandas as pd
from numpy import nan
import requests as rq
import csv
import re

uber = pd.read_csv("/Uber.csv")

In [ ]:
# Функция для вывода некоторой информации о значениях в столбце (строкового типа)
def print_column_value_info(table: pd.DataFrame, column: str):
    print(f"Число значений с учетом регистра: {table[column].unique().size}")
    print(f"Число значений без учета регистра: {table[column].str.strip().str.lower().unique().size}")
    print(f"Число NaN значений: {table[table[column].isna()][column].size}")
    print(f"Число пустых значений: {table[table[column].str.strip() == ''][column].size}")

# Столбец Date

In [ ]:
# Проверка, есть ли незаполненные значения
print_column_value_info(uber, "Date")

Число значений с учетом регистра: 92
Число значений без учета регистра: 92
Число NaN значений: 0
Число пустых значений: 0


In [ ]:
# Все значения преобразуются к типу datetıme, значит, ничего менять не нужно
pd.to_datetime(uber["Date"])

,Date
0,2014-07-06
1,2014-07-04
2,2014-07-05
3,2014-07-06
4,2014-07-05
...,...
194987,2014-09-28
194988,2014-09-28
194989,2014-09-28
194990,2014-09-28


 # Столбец Time

In [ ]:
# Проверка, есть ли в столбце пустые значения или значения, не соответствующие формату времени
empty_count = uber[uber['Time'].isnull()].size
wrong_format_count = uber.size - uber[uber['Time'].str.match('\d\d?:\d\d')].size
print(f"Число незаполненных ячеек: {empty_count}")
print(f"Число ячеек, не соответствующих формату: {wrong_format_count}")

Число незаполненных ячеек: 0
Число ячеек, не соответствующих формату: 0


In [ ]:
# Приведение столбца Time к float
def string_timedelta_to_float(str_td: str):
    ms = str_td.split(":")
    min = int(ms[0])
    sec = int(ms[1])
    if sec != 0:
        min += 60 / sec
    return min

uber["Time"] = uber["Time"].apply(lambda str_time: string_timedelta_to_float(str_time))

# Cтолбец PuFrom

In [ ]:
print_column_value_info(uber, "PuFrom")

Число значений с учетом регистра: 1265
Число значений без учета регистра: 719
Число NaN значений: 37030
Число пустых значений: 305


In [ ]:
# Приведение столбца PuFrom к верхнему регистру
uber["PuFrom"] = uber["PuFrom"].str.strip().str.upper()

In [ ]:
# Пустые значения (состоящие из пробелов) заменим на NaN для однородности
uber.loc[uber["PuFrom"].str.strip() == "","PuFrom"] = nan

In [ ]:
print_column_value_info(uber, "PuFrom")

Число значений с учетом регистра: 718
Число значений без учета регистра: 718
Число NaN значений: 37335
Число пустых значений: 0


# Cтолбец State

In [ ]:
print_column_value_info(uber, "State")

Число значений с учетом регистра: 1001
Число значений без учета регистра: 793
Число NaN значений: 0
Число пустых значений: 0


In [ ]:
# Приведение столбца State к верхнему регистру
uber["State"] = uber["State"].str.strip().str.upper()

In [ ]:
print_column_value_info(uber, "State")

Число значений с учетом регистра: 793
Число значений без учета регистра: 793
Число NaN значений: 0
Число пустых значений: 0


Если State - это столбец для штата, то вариантов значений в нем многовато.

In [ ]:
uber[["State", "PuFrom"]].tail(20)

,State,PuFrom
194972,NY,MANHATTAN
194973,NJ,JERSEY CITY
194974,NY,MANHATTAN
194975,NY,MANHATTAN
194976,NY,MANHATTAN
194977,LGA DELTA PICK-UP ISLAND,NaN
194978,LGA WESTJET PICK-UP ISLAND,NaN
194979,EWR UNITED TERM C AREA 3,NaN
194980,LGA JETBLUE PICK-UP ISLAND,NaN
194981,NY,MANHATTAN


Судя по тому, как выглядят некоторые значения, им следует быть в столбце PuFrom (pick up from), который в этом случае обычно как раз не заполнен.

In [ ]:
# Первые 3 символа State, когда PuFrom пуст
uber[uber["PuFrom"].isna()]["State"].str[0:3].value_counts()

,count
State,
JFK,15465
LGA,13684
EWR,8060
HPN,62
TEB,56
M,7
ISP,1


Первые пять значений в таблице выше - международные обозначения аэропортов в США. Мне кажется правильным переместить эти значения в столбец PuFrom, а State указать название штата, в котором находится аэропорт.

В таблице нет строк, которые начинаются на эти сочетания букв и не относятся к аэропортам.

In [ ]:
ap_code_state = {
    "JFK": "NY",
    "LGA": "NY",
    "EWR": "NJ",
    "HPN": "NY",
    "TEB": "NJ"
}

In [ ]:
# Значения столбца PuFrom, когда в State содержится информация об аэропорте
# Убедимся, что не потеряем важную информацию при изменении столбцов
uber[uber["State"].str[0:3].isin(ap_code_state.keys())]["PuFrom"].unique()

array(['EWR', nan, 'JFK', 'LGA', 'NWK', 'LAG', 'MANHATTAN', 'UN',
       'WHIPPLEVILLE', 'TETERBORO'], dtype=object)

In [ ]:
# Выполнение преобразований
rows_to_change = uber[uber["State"].str[0:3].isin(ap_code_state.keys())]
uber.loc[rows_to_change.index, "PuFrom"] = rows_to_change["State"]
uber.loc[rows_to_change.index, "State"] = rows_to_change["State"].str[0:3].replace(ap_code_state)

In [ ]:
# Проверим, как преобразования отразились на значениях в столбцах
print_column_value_info(uber, "PuFrom")

Число значений с учетом регистра: 1494
Число значений без учета регистра: 1494
Число NaN значений: 8
Число пустых значений: 0


In [ ]:
print_column_value_info(uber, "State")

Число значений с учетом регистра: 10
Число значений без учета регистра: 10
Число NaN значений: 0
Число пустых значений: 0


In [ ]:
uber["State"].unique()

array(['NY', 'NJ', 'CT', 'PA', 'RI', 'M', 'ISP', 'VA', 'DC', 'MA'],
      dtype=object)

In [ ]:
# Нет таких названий штатов, которые сокращаются как M или ISP
# Возможно, такие строки нужно удалить или отредактировать
uber[uber["State"].isin(["M", "ISP"])]

,Date,Time,State,PuFrom,Address,Street
16422,2014.08.12,7.333333,M,NaN,45,WALL ST
36078,2014.07.31,12.000000,M,NaN,145,W 45 ST
41371,2014.07.28,16.000000,M,NaN,1965,BROADWAY
42270,2014.07.22,9.000000,M,NaN,541,LEXINGTON AVE
43726,2014.09.30,16.000000,M,NaN,450,W 33 ST
83754,2014.09.23,21.000000,ISP,NaN,,
107578,2014.07.26,16.000000,M,NaN,118,W 13 ST
168746,2014.08.21,18.000000,M,NaN,320,W 66 ST


In [ ]:
# Удалим строку, где State = ISP, т.к. в ней нет никакой информации о маршруте
# cust = cust.drop([54], axis=0).reset_index(drop=True)
idx_to_drop = uber[uber["State"] == "ISP"].index
uber = uber.drop(idx_to_drop, axis="rows").reset_index(drop=True)

Google подсказал, что все адреса, где State = M, расположены на Манхэттене в Нью-Йорке. Время поездки для каждого из них довольно короткое, можно предположить, что перемещения были в пределах этого района. Тогда State можно изменить на NY и PuFrom на MANHATTAN.

In [ ]:
rows_to_change = uber[uber["State"] == "M"]
uber.loc[rows_to_change.index, "State"] = "NY"
uber.loc[rows_to_change.index, "PuFrom"] = "MANHATTAN"

In [ ]:
uber["State"].unique()

array(['NY', 'NJ', 'CT', 'PA', 'RI', 'VA', 'DC', 'MA'], dtype=object)

In [ ]:
# Приятно, что после очистки все значения столбца PuFrom теперь заполнены
print_column_value_info(uber, "PuFrom")

Число значений с учетом регистра: 1493
Число значений без учета регистра: 1493
Число NaN значений: 0
Число пустых значений: 0


# Столбец Street

In [ ]:
print_column_value_info(uber, "Street")

Число значений с учетом регистра: 9566
Число значений без учета регистра: 8523
Число NaN значений: 0
Число пустых значений: 54147


In [ ]:
# Приведение значений к верхнему регистру
uber["Street"] = uber["Street"].str.strip().str.upper()

In [ ]:
uber["Street"].value_counts().sort_index()

,count
Street,
,54147
.AMSTERDAM AVE,1
.COMMUNITY DR,1
0,1
1,3
...,...
ZABRISKIE ST,1
ZELLER DR,1
ZOLA ST,1


In [ ]:
# Строки, имеющие посторонние символы в начале
not_empty_street_rows = uber[uber["Street"] != ""]
not_empty_street_rows[~not_empty_street_rows["Street"].str[0].str.isalnum()]

,Date,Time,State,PuFrom,Address,Street
13516,2014.07.15,4.142857,NY,MANHATTAN,509,.AMSTERDAM AVE
77030,2014.09.23,21.000000,NY,MANHATTAN,485,]LEXINGTON AVE
112695,2014.09.21,6.000000,NY,MANHASSET,300,.COMMUNITY DR
156196,2014.08.02,22.000000,NJ,FORT LEE,,????


In [ ]:
# Замена значения ???? на пустое
uber.loc[uber[uber["Street"].str.contains("\?")].index, "Street"] = ""

In [ ]:
# Удаление посторонних символов в начале некоторых строк
not_empty_street_rows = uber[uber["Street"] != ""]
rows_to_change = not_empty_street_rows[~not_empty_street_rows["Street"].str[0].str.isalnum()]
uber.loc[rows_to_change.index, "Street"] = uber.iloc[rows_to_change.index]["Street"].str[1:]

In [ ]:
# Строки, имеющие посторонние символы в конце
not_empty_street = uber[uber["Street"] != ""]
not_empty_street[~not_empty_street["Street"].str[-1].str.isalnum()]

,Date,Time,State,PuFrom,Address,Street
645,2014.09.17,6.000000,NY,MELVILLE,169,ALTESSA BLVD.
12769,2014.07.03,27.000000,NY,OCEANSIDE,3050,OCEANSIDE RD.
23881,2014.07.31,18.000000,NY,PLAINVIEW,303,SUNNYSIDE BLVD.
24724,2014.08.13,6.000000,NY,HARTSDALE,50,E.HARTSDALE AVE.
24986,2014.09.06,10.304348,NJ,BAYONNE,14,PORT TERMINAL BLVD.
...,...,...,...,...,...,...
174953,2014.09.13,18.000000,NJ,JERSEY CITY,837,JERSEY AVE.
180924,2014.07.06,21.000000,NJ,TETERBORO,101,CHARLES LINDBERG DR.
183192,2014.09.06,24.333333,NY,GREAT NECK,555,NORTHERN BLVD.
186073,2014.09.21,15.000000,NY,PORT CHESTER,18,MILL ST.


In [ ]:
# Удаление посторонних символов в конце некоторых строк
not_empty_street_rows = uber[uber["Street"] != ""]
rows_to_change = not_empty_street_rows[~not_empty_street_rows["Street"].str[-1].str.isalnum()]
uber.loc[rows_to_change.index, "Street"] = uber.iloc[rows_to_change.index]["Street"].str[0:-1]

Я заметила, что где-то улицы обозначены с сокращением, где-то без (ST - STREET и т.п.). Я собираюсь привести все названия к одному виду, используя общепринятые почтовые сокращения (https://pe.usps.com/text/pub28/28apc_002.htm).

Также к названию улицы в конце или начале может быть добавлено направление (EAST - E и т.д.). Их я тоже хочу привести к одному формату.

Я планировала использовать библиотеку address, но она не работает с Python3. В исходниках этой библиотеки на GitHub есть файл csv со всеми вариантами обозначения улиц (https://github.com/SwoopSearch/pyaddress/blob/master/address/suffixes.csv), его я и буду использовать.

При изменении нужно обращать внимание, что подстрока, которую нужно изменить, является отдельным словом. Например, строка BROADWAY содержит ROAD, но не должна быть изменена. Также считаем, что сторона света может быть указана только в начале или конце строки, другие совпадения учитываться не будут.

Можно заметить, что некоторые строки содержат точки, при этом в столбце есть точно такие же строки без точки, поэтому имеет смысл ее удалить совсем.

В названиях некоторых улиц есть слово "святой", которое обозначается как ST или SAINT. Для обнородности имеет смысл SAINT заменить на ST (наоборот не получится из-за конфликта с сокращением STREET).

In [ ]:
# Формирование словаря со всеми обозначениями улиц
resp = rq.get("https://raw.githubusercontent.com/SwoopSearch/pyaddress/master/address/suffixes.csv")
suffixes = {}
reader = csv.reader(resp.text.splitlines())
for row in reader:
    suffixes[row[0].strip()] = row[1].strip()

In [ ]:
# Формирование словаря со всеми направлениями
directions = {'N': "NORTH",
              'S': "SOUTH",
              'E': "EAST",
              'W': "WEST",
              'NE': "NORTHEAST",
              'NW': "NORTHWEST",
              'SE': "SOUTHEAST",
              'SW': "SOUTHWEST"
              }

In [ ]:
# Функция преобразует названия улиц к общему виду
# Заодно убирает точки и лишние пробелы между словами
def format_street(street: str) -> str:
    words = re.split("\.| ", street)
    # Удаление пустых строк, попавших в список
    while "" in words:
        words.remove("")
    # Если строка пустая или состоит из пробелов и точек
    if not words:
        return ""
    # Преобразование обозначений улиц, замена SAINT на ST
    for idx in range(0, len(words)):
        if words[idx] == "SAINT":
            words[idx] = "ST"
        elif words[idx] in suffixes:
            words[idx] = suffixes[words[idx]]
    # Если в начале или конце строки сокращенное обозначение направления, оно преобразуется в полное
    for i in range(-1, 1):
        if words[i] in directions:
            words[i] = directions[words[i]]
    return " ".join(words)

In [ ]:
# Преобразование данных в столбце
uber["Street"] = uber["Street"].apply(lambda s: format_street(s))

In [ ]:
# Уникальных значений стало поменьше
print_column_value_info(uber, "Street")

Число значений с учетом регистра: 7911
Число значений без учета регистра: 7911
Число NaN значений: 0
Число пустых значений: 54148


# Столбец Address

In [ ]:
print_column_value_info(uber, "Address")

Число значений с учетом регистра: 4328
Число значений без учета регистра: 4328
Число NaN значений: 0
Число пустых значений: 54396


In [ ]:
uber["Address"].value_counts().sort_index()

,count
Address,
,54396
-,1
0,1
0-46,2
1,2018
...,...
N546BW,1
PVT,1
TAIL #,1


In [ ]:
uber["Address"] = uber["Address"].str.strip().str.upper()

# Удаление некорректных строк

Пустые значения могут содержать только столбцы Address и Street. При этом если не заполнен только Address, то такая строка вполне корректна. В нее можно записать какое-то значение по умолчанию, например, "-".

In [ ]:
# Число строк, где Street не пуст, а Address пуст
uber[(uber["Street"] != "") & (uber["Address"] == "")].index.size

248

In [ ]:
# Устанавливаем значение по умолчанию для Address в таких строках
rows_to_change = uber[(uber["Street"] != "") & (uber["Address"] == "")]
uber.loc[rows_to_change.index, "Address"] = "-"

In [ ]:
# Число строк, где Street пуст, а Address не пуст
uber[(uber["Street"] == "") & (uber["Address"] != "")].index.size

0

In [ ]:
# Число строк, где Street и Address пусты одновременно
uber[(uber["Street"] == "") & (uber["Address"] == "")].index.size

54148

Предположим, что для некоторой задачи нам нужны только корректные заполненные данные. Тогда соберем в новый датасет только строки, которые:
*   содержат только непустые данные;
*   не дублируются (хотя для этого датасета дублирование может быть оправданным);
*   время поездки > 0.

In [ ]:
# Новый DataFrame, в котором только полностью заполненные строки
uber2 = uber[(uber["Street"] != "") & (uber["Address"] != "")].reset_index(drop=True)

In [ ]:
# Удаление полных дубликатов
uber2 = uber2.drop(uber2[uber2.duplicated()].index, axis="rows").reset_index(drop=True)

In [ ]:
# Удаление строк, в которых время поездки = 0
uber2 = uber2.drop(uber2[uber2["Time"] == 0].index, axis=0).reset_index(drop=True)

In [ ]:
uber2

,Date,Time,State,PuFrom,Address,Street
0,2014.07.06,16.000000,NY,MANHATTAN,50,MURRAY ST
1,2014.07.04,11.000000,NY,MANHATTAN,143,AVE B
2,2014.07.05,6.333333,NY,MANHATTAN,125,CHRISTOPHER ST
3,2014.07.06,6.000000,NY,MANHATTAN,217,EAST 7 ST
4,2014.07.05,8.333333,NY,MANHATTAN,521,WEST 26 ST
...,...,...,...,...,...,...
138342,2014.09.29,9.000000,NY,MANHATTAN,9,WEST 57 ST
138343,2014.09.29,6.000000,NJ,RIDGEWOOD,756,BELMONT RD
138344,2014.09.28,7.000000,NY,MANHATTAN,112,WEST 78 ST
138345,2014.09.28,17.000000,NY,MANHATTAN,601,WEST 57 ST
